In [20]:
import os

In [21]:
os.environ['LANGCHAIN_TRACING_V2'] = "True"
os.environ['LANGCHAIN_API_KEY']= "ls__6cc44c98e5254ca0a72a19de931a7916"

In [22]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from services import create_chroma_client
from langchain_community.chat_models import ChatOllama

In [23]:
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
import re
from typing import Union, Optional, Type

from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
)
from langchain_core.agents import AgentAction, AgentFinish

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool

In [24]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [25]:
from langchain.globals import set_debug

set_debug(True)

In [26]:
oembed = OllamaEmbeddings(model="openchat")
db = Chroma(client=create_chroma_client(), embedding_function=oembed, collection_name="news")

In [27]:
def retrieve(tematica: str, fecha: str):
    import time
    import datetime as dt

    fecha_dt = dt.datetime.today()
    if fecha == "ayer" or fecha == 'yesterday':
        fecha_dt = dt.datetime.today() - dt.timedelta(days=1)

    fecha_unix = time.mktime(fecha_dt.timetuple())
    documents = db.similarity_search(tematica, k=10, filter={'published': {'$gt': fecha_unix}})

    if len(documents) > 0:
        return [document.page_content for document in documents]
    return []

In [28]:
class NewsInput(BaseModel):
    tematica: str = Field(description="tematica a buscar en las noticias")
    fecha: str = Field(description="fecha a buscar las noticias")


class NewsTool(BaseTool):
    name = "Buscar_noticias_por_tematica_y_fecha"
    description = "Permite obtener las ultimas noticias por una tematica y fecha dada"
    args_schema: Type[BaseModel] = NewsInput

    def _run(
        self,
        tematica: str,
        fecha: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        return retrieve(tematica, fecha)

    async def _arun(
        self,
        tematica: str,
        fecha: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        return retrieve(tematica, fecha)

In [29]:
llm = ChatOllama(
    model="mistral",
    temperature=0,
    streaming=True,
)
llm_with_stop = llm.bind(stop=["\nObservacion", "\nObservación"])
tools = [NewsTool()]

In [30]:
def render_tools(tools):
    tool_strings = []
    for tool in tools:
        args_schema = ", ".join(tool.args.keys())
        tool_strings.append(f"{tool.name}: {tool.description}. Argumentos: {args_schema}")
    return "\n".join(tool_strings)

In [31]:
template = (
    """Eres un agente que debe responder preguntas utilizando noticias.
Necesitas determinar la tematica que plantea el usuario
y la fecha con la que buscar las noticias. Si no se especifica una fecha, entonces
debes asumir "ayer".

Tenes acceso a las siguientes herramientas que te permitiran responder a las preguntas:

"""
    + render_tools(tools)
    + """

Tenes que usar el siguiente formato:

```
Pregunta: pregunta que debes responder
Pensamiento: siempre tienes que pensar qué hacer
Acción: la acción a tomar que debes hacer para responder. debe ser alguna de las herramientas ["""
    + ", ".join([tool.name for tool in tools])
    + """], con los argumentos entre parentesis, con el formato nombre_argumento="valor_argumento" y separados por coma. Por ejemplo, herramienta(argumento1=valor1).
Observación: el resultado de la accion
... (este proceso de Pensamiento/Acción/Observación se puede repetir N veces)
Pensamiento: ahora se la respuesta final.
Respuesta final: la respuesta final a la pregunta original.
```

Pregunta: {input}

{agent_scratchpad}"""
)

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("user", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [33]:
from typing import List, Tuple
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents.format_scratchpad import format_log_to_messages

In [34]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Respuesta final:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Respuesta final:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Acci[o|ó]n: (\w+)\((.*)\)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        pares = action_input.replace('"', '').split(', ')
        diccionario = {}
        for par in pares:
            clave, valor = par.split('=')
            diccionario[clave] = valor

        return AgentAction(
            tool=action,
            tool_input=diccionario,
            log=llm_output,
        )

In [35]:
def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [36]:
agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]) if x.get("chat_history") else [],
        "agent_scratchpad": lambda x: format_log_to_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | CustomOutputParser()
)

In [37]:
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(agent=agent, tools=tools).with_types(input_type=AgentInput)

In [38]:
print(agent_executor.invoke({'input': "que pasó ayer en gran hermano?"}))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "que pasó ayer en gran hermano?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,chat_history,agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,chat_history,agent_scratchpad> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,chat_history,agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,chat_history,agent_scratchpad> > 4:chain:RunnableLambda] s] Exiting Chain run with output: